In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install facenet_pytorch

     |████████████████████████████████| 1.9MB 7.7MB/s 


In [ ]:
import torch
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from facenet_pytorch import MTCNN
import cv2
from tqdm import tqdm
import os
import copy
import math

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [ ]:
if not os.path.exists('/content/tempOutput/'): # model makes use of this as sort of temporary
    os.makedirs('/content/tempOutput/')         # storage (stores extracted frames from video before run prediction on them)

**direct unzip from drive to colab. (no need to copy first to colab and then unzipping)**

In [ ]:
!unzip '/content/gdrive/MyDrive/optionB_blah_testSet/optionB_blah_testSet.zip' -d '/content/'

**rename the folder here, instead of changing it everywhere in the code (lol)**

In [ ]:
os.rename('/content/optionB_blah_testSet', '/content/unprocessedTestData')

# json file containing truth label aese use krna

---



In [ ]:
import json
path = '/content/unprocessedTestData/'
fp=open(path+"metadata.json",)
data=json.load(fp)
#print('The true label is:',data[name]['label'])   # where name is string containing just name(not path) of the video



# d="/content/testSet/unprocessedTestData"    # folder where test videos are located
#for video in os.listdir(d):   # this for loop will give video names one by one

    # path_to_video = d+'/'+video    # give path to functing in which video is tested by model
    # label = data[video]['label']   # give truth label from json


In [ ]:
def load_checkpoint(checkpoint_path):
    checkpoint=torch.load(checkpoint_path)
    model_conv=torchvision.models.resnet50(pretrained=True)

    for param in model_conv.parameters():
        param.requires_grad=False

    num_ftrs=model_conv.fc.in_features
    model_conv.fc=nn.Linear(num_ftrs,2)

    #loading the weights
    model_conv.load_state_dict(checkpoint['model_state_dict'],strict=False)
    model_conv.eval()
    return model_conv

In [ ]:
def applyTransforms(inp):
    outp = transforms.functional.resize(inp, [224,224])
    outp = transforms.functional.to_tensor(outp)
    outp = transforms.functional.normalize(outp, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    return outp

**extract every 30th frame of video, extract face from those frames, feed them into network**

In [ ]:

# function that :
# splits passed video into frames and identifies faces within the frames
# does forward propogation for all these face frames through the model loaded with trained weights
# each forward propogation results in a probability for the frame being "fake" or "real"
# function returns the average probability and percentage of all the frames being real and fake
# class with highest probability is considered the detected label of the video

def myVideo(file_name,model):
    mtcnn = MTCNN(margin=20, keep_all=True, post_process=False, device=device)
    # Load video
    v_cap = cv2.VideoCapture(file_name)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Loop through video, taking some no of frames to form a batch   (here, every 30th frame)
    frames = []
    for i in tqdm(range(v_len)):

        # Load frame
        success = v_cap.grab()
        if i % 30 == 0:
            success, frame = v_cap.retrieve()
        else:
            continue
        if not success:
            continue

        # Add to batch
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(Image.fromarray(frame))



    #detect faces in frames &  saving frames to file
    f="/content/tempOutput/"
    frames_paths = [f+'image' + str(i) +'.jpg' for i in range(len(frames))]
    faces = mtcnn(frames,save_path=frames_paths)

    path=os.listdir(f)
    vals_f=[]
    vals_r=[]
    for x in path:
        img = Image.open(f + x)
        imageTensor = applyTransforms(img)
        minibatch = torch.stack([imageTensor])
        #model_conv(minibatch)
        softMax = nn.Softmax(dim = 1)
        preds = softMax(model(minibatch))
        vals_f.append(preds[0,0].item())
        vals_r.append(preds[0,1].item())

    av_f=sum(vals_f)/len(path)
    print("average probability of fakeness:",av_f)
    print('Percentage of fakeness: {:.4f}'.format(av_f*100))

    av_r=sum(vals_r)/len(path)
    print("average probability of realness:",av_r)
    print('Percentage of realness: {:.4f}'.format(av_r*100))



    return av_f

In [ ]:
#import os
def testing(path,model):

    #remove previous temp output if any
    f="/content/tempOutput/"
    reqd=os.listdir(f)

    if len(reqd)!=0:
        for i in reqd:
            os.remove(f+i)


    prediction = myVideo(path,model) # returns probability of fakeness of video



    return prediction

In [ ]:
#  Call load_checkpoint() function ( to load the trained weights into the ResNet-50 model )
model_conv=load_checkpoint("/content/gdrive/MyDrive/optionB_weights1/finalCheckpoint_tensor(0.9656, device='cuda:0', dtype=torch.float64)_0.10349578748715126.pth")

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
#########   1 for real
#########   0 for fake
path = '/content/unprocessedTestData/'
nTotal=0
nTotalCorrect=0
nTotalIncorrect=0
for f in os.listdir(path):
    if f == 'metadata.json':
        continue

    pred = testing(path+f,model_conv)

    pred = (pred<0.5)*1 # 1 when real (fake probability less than 0.5, therefore its real and therefore label 1)
    truth = (data[f]['label'] == 'REAL') * 1 #  1 when label is real, 0 for fake

    print(data[f]['label'])
    print(truth)
    if (pred==truth):
        print("\nCorrect Prediction!!")
        print(pred)

        nTotalCorrect+=1
    else:
        print("\nIncorrect.")
        print(pred)
        nTotalIncorrect+=1

    nTotal+=1
    print()

    print("Total Correct: "+str(nTotalCorrect))
    print("Total Incorrect: "+str(nTotalIncorrect))
    print("Total: "+str(nTotal))
    print("-----------------------------\n")




100%|██████████| 300/300 [00:02<00:00, 120.67it/s]
/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/utils/detect_face.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  batch_boxes, batch_points = np.array(batch_boxes), np.array(batch_points)
/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/mtcnn.py:339: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  boxes = np.array(boxes)
/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/mtcnn.py:340: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (

average probability of fakeness: 0.0696802470957454
Percentage of fakeness: 6.9680
average probability of realness: 0.9303197677318866
Percentage of realness: 93.0320
REAL
1

Correct Prediction!!
1

Total Correct: 1
Total Incorrect: 0
Total: 1
-----------------------------



  2%|▏         | 7/300 [00:00<00:04, 60.47it/s]

average probability of fakeness: 0.9924466063578924
Percentage of fakeness: 99.2447
average probability of realness: 0.0075533990557232755
Percentage of realness: 0.7553
FAKE
0

Correct Prediction!!
0

Total Correct: 2
Total Incorrect: 0
Total: 2
-----------------------------



  3%|▎         | 9/300 [00:00<00:03, 80.95it/s]

...

average probability of fakeness: 0.08209926697732457
Percentage of fakeness: 8.2099
average probability of realness: 0.9179007302631031
Percentage of realness: 91.7901
REAL
1

Correct Prediction!!
1

Total Correct: 190
Total Incorrect: 8
Total: 198
-----------------------------



  2%|▏         | 5/300 [00:00<00:06, 48.76it/s]

average probability of fakeness: 0.9540669255786471
Percentage of fakeness: 95.4067
average probability of realness: 0.04593306562552849
Percentage of realness: 4.5933
FAKE
0

Correct Prediction!!
0

Total Correct: 191
Total Incorrect: 8
Total: 199
-----------------------------



100%|██████████| 300/300 [00:03<00:00, 92.37it/s]


average probability of fakeness: 0.9882213672002157
Percentage of fakeness: 98.8221
average probability of realness: 0.01177863659298358
Percentage of realness: 1.1779
FAKE
0

Correct Prediction!!
0

Total Correct: 192
Total Incorrect: 8
Total: 200
-----------------------------



In [ ]:
print((data['idzntwkkjy.mp4']['label']=='REAL')*1)

In [ ]:
import cv2
cv2.imread('/content/tempOutput/image0.jpg').shape

(160, 160, 3)